In [2]:
import librosa
import librosa.display
from librosa.core import load
import numpy as np
import pandas as pd
import glob
import os, sys
import time

### Load Data

In [3]:
VERSION = 4 
df_joint_train_org   = pd.read_csv(f'./features/cache_all_features_train_V{VERSION}.csv')
df_joint_train_org   = df_joint_train_org.drop(columns=['GNE_max_gne','GNE_mean_gne','GNE_stddev_gne','GNE_sum_gne'])                         
df_joint_test_org = pd.read_csv(f'./features/cache_all_features_test_V{VERSION}.csv').drop(
    columns=['GNE_max_gne','GNE_mean_gne','GNE_stddev_gne','GNE_sum_gne'])
feature_column_names = [i for i in df_joint_train_org.columns \
                        if i not in ['file_path','renamed_file_path','split','sentiment_value','emotional_category']]
print("shape of train set original: ", df_joint_train_org.shape)
print("shape of test  set original: ", df_joint_test_org.shape)

df_joint_train_ds  = pd.read_csv(f'./features/cache_train_V4_resampled_2500.csv')
df_joint_test_ds = pd.read_csv(f'./features/cache_test_V4_resampled_250.csv')
print("shape of train set 3cls balanced after downsample: ", df_joint_train_ds.shape)
print("shape of test  set 3cls balanced after downsample: ", df_joint_test_ds.shape)

df_joint_train_aug  = pd.read_csv(f'./features/cache_train_V4_augmented.csv')
print("shape of train set 3cls balanced after augmentation: ", df_joint_train_aug.shape)

feat_rank_emo = pd.read_csv('./features/relationship_study/relationship_stats_test_result_emotion_prosody_V4_augmented.csv')
feat_rank_sti = pd.read_csv('./features/relationship_study/relationship_stats_test_result_sentiment_prosody_V4_augmented.csv')
feat_rank_emo.correlation_coeff = abs(feat_rank_emo.correlation_coeff)
feat_rank_sti.correlation_coeff = abs(feat_rank_sti.correlation_coeff)
feature_types = ['mfcc','chroma','mel','zcr','rms','Intensity','Pitch','GNE','HNR','Spectrum','Formant']

shape of train set original:  (10981, 1546)
shape of test  set original:  (1180, 1546)
shape of train set 3cls balanced after downsample:  (6538, 1546)
shape of test  set 3cls balanced after downsample:  (680, 1546)
shape of train set 3cls balanced after augmentation:  (24885, 1546)


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [4]:
def get_X_n_y_by_cond(df,cols):
    X = df[cols]
    y_s,y_e = df['sentiment_value'],df['emotional_category']
    return X , y_s, y_e

X_train_org, y_train_s_org, y_train_e_org = get_X_n_y_by_cond(df_joint_train_org,feature_column_names)
X_test_org, y_test_s_org, y_test_e_org = get_X_n_y_by_cond(df_joint_test_org,feature_column_names)
X_train_aug, y_train_s_aug, y_train_e_aug = get_X_n_y_by_cond(df_joint_train_aug,feature_column_names)
X_train_ds, y_train_s_ds, y_train_e_ds = get_X_n_y_by_cond(df_joint_train_ds,feature_column_names)
X_test_ds,  y_test_s_ds, y_test_e_ds = get_X_n_y_by_cond(df_joint_test_ds,feature_column_names)

### Check feature ranking

In [5]:
feat_rank_emo.describe()

,p_value,is_significant,f_score,correlation_coeff,mutual_info_score
count,52.000000,52.0,52.000000,52.000000,52.000000
mean,0.000002,1.0,273.944456,0.076302,0.110419
std,0.000014,0.0,333.344792,0.061493,0.085777
min,0.000000,1.0,4.374100,0.004200,0.008500
25%,0.000000,1.0,30.649250,0.030450,0.034975
50%,0.000000,1.0,119.646850,0.057250,0.088650
75%,0.000000,1.0,367.601250,0.101250,0.186450
max,0.000100,1.0,1136.014000,0.265500,0.296800


In [7]:
features_score_above_p50 = feat_rank_emo[(feat_rank_emo.f_score>119.647) &
(feat_rank_emo.mutual_info_score>0.057)&
(feat_rank_emo.correlation_coeff>0.0885)]['feature'].values

print("features with score above p50 from Emotion label: ", len(features_score_above_p50))
print(features_score_above_p50)

features with score above p50 from Emotion label:  15
['Pitch_q3_pitch' 'Pitch_mean_pitch' 'Intensity_q3_intensity'
 'Pitch_mean_absolute_pitch_slope'
 'Pitch_pitch_slope_without_octave_jumps' 'Intensity_mean_intensity'
 'Pitch_stddev_pitch' 'Intensity_q1_intensity'
 'Intensity_median_intensity' 'Pitch_max_pitch' 'Intensity_min_intensity'
 'Intensity_stddev_intensity' 'Spectrum_skewness_spectrum'
 'Spectrum_kurtosis_spectrum' 'Spectrum_stddev_spectrum']


In [8]:
feat_rank_emo[feat_rank_emo.feature.str.contains('Pitch')]

,feature,p_value,is_significant,f_score,correlation_coeff,mutual_info_score
0,Pitch_q3_pitch,0.0,1,1136.0140,0.1205,0.2968
1,Pitch_mean_pitch,0.0,1,999.4002,0.0978,0.2960
3,Pitch_mean_absolute_pitch_slope,0.0,1,969.5565,0.2655,0.2191
5,Pitch_pitch_slope_without_octave_jumps,0.0,1,924.0906,0.2105,0.1926
7,Pitch_stddev_pitch,0.0,1,701.6538,0.1855,0.1976
8,Pitch_median_intensity,0.0,1,690.3525,0.0552,0.2321
11,Pitch_max_pitch,0.0,1,549.7149,0.1858,0.2241
12,Pitch_q1_pitch,0.0,1,412.6626,0.0192,0.1844
20,Pitch_min_pitch,0.0,1,180.2162,0.0183,0.1293
26,Pitch_voiced_fraction,0.0,1,117.9129,0.1926,0.1084


In [9]:
feat_rank_sti.describe()

,p_value,is_significant,f_score,correlation_coeff,mutual_info_score
count,52.000000,52.0,52.000000,52.000000,52.000000
mean,0.000010,1.0,507.117058,0.133988,0.056919
std,0.000057,0.0,805.488797,0.128863,0.060862
min,0.000000,1.0,7.782200,0.005400,0.000000
25%,0.000000,1.0,31.890575,0.044200,0.011225
50%,0.000000,1.0,126.505350,0.073600,0.032450
75%,0.000000,1.0,591.604775,0.195025,0.085425
max,0.000400,1.0,3284.645300,0.445700,0.240300


In [12]:
features_score_above_p50 = feat_rank_sti[(feat_rank_sti.f_score>126.5) & 
(feat_rank_sti.correlation_coeff>0.0736) & 
(feat_rank_sti.mutual_info_score>0.0324)]['feature'].values 

print("features with score above p50 from Sentiment label: ", len(features_score_above_p50))
print(features_score_above_p50)

features with score above p50 from Sentiment label:  23
['Pitch_q3_pitch' 'Pitch_mean_pitch'
 'Pitch_pitch_slope_without_octave_jumps' 'Pitch_median_intensity'
 'Pitch_mean_absolute_pitch_slope' 'Pitch_stddev_pitch' 'Pitch_max_pitch'
 'Intensity_max_intensity' 'Pitch_q1_pitch' 'Intensity_q3_intensity'
 'Spectrum_band_density_difference' 'Spectrum_band_energy_difference'
 'Spectrum_center_of_gravity_spectrum' 'Formant_f1_median'
 'Formant_f1_mean' 'Intensity_relative_max_intensity_time'
 'Pitch_min_pitch' 'Spectrum_skewness_spectrum'
 'Spectrum_kurtosis_spectrum' 'Intensity_median_intensity'
 'Intensity_mean_intensity' 'Spectrum_band_density' 'Spectrum_band_energy']


### import Model

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score, precision_recall_fscore_support

dt_clf= DecisionTreeClassifier()
rf_clf = RandomForestClassifier()

def try_clf_with_feature_selected(clf_model,X_train,X_test,y_train,y_test):
    predictions = clf_model.fit(X_train, y_train).predict(X_test)
    print(classification_report(y_test, predictions))
    print(confusion_matrix(y_test, predictions))

In [19]:
global scenario_sets 
scenario_sets = {
    'aug-org-emo': [X_train_aug,X_test_org,y_train_e_aug,y_test_e_org],
    'aug-ds-emo': [X_train_aug,X_test_ds,y_train_e_aug,y_test_e_ds],
    'aug-org-sti': [X_train_aug,X_test_org,y_train_s_aug,y_test_s_org],
    'aug-ds-sti': [X_train_aug,X_test_ds,y_train_s_aug,y_test_s_ds],
    'ds-org-emo': [X_train_ds,X_test_org,y_train_e_ds,y_test_e_org],
    'ds-ds-emo': [X_train_ds,X_test_ds,y_train_e_ds,y_test_e_ds],
    'ds-org-sti': [X_train_ds,X_test_org,y_train_s_ds,y_test_s_org],
    'ds-ds-sti': [X_train_ds,X_test_ds,y_train_s_ds,y_test_s_ds],
    'org-org-emo': [X_train_org,X_test_org,y_train_e_org,y_test_e_org],
    'org-ds-emo': [X_train_org,X_test_ds,y_train_e_org,y_test_e_ds],
    'org-org-sti': [X_train_org,X_test_org,y_train_s_org,y_test_s_org],
    'org-ds-sti': [X_train_org,X_test_ds,y_train_s_org,y_test_s_ds]
}
def experiment(clf_model,trainset_type,testset_type,features,cls_type):
    key_comb = '-'.join([trainset_type,testset_type,cls_type])
    X_train, X_test, y_train, y_test = scenario_sets[key_comb]
    X_train, X_test = X_train[features], X_test[features]
    y_pred = clf_model.fit(X_train, y_train).predict(X_test)
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

In [20]:
experiment(rf_clf,'org','org',features_score_above_p50,'emo')

              precision    recall  f1-score   support

       Anger       0.69      0.76      0.72       187
    Calmness       0.46      0.81      0.59        16
     Disgust       0.49      0.48      0.49       187
        Fear       0.58      0.48      0.53       187
   Happiness       0.60      0.43      0.50       187
  Neutrality       0.57      0.59      0.58       167
     Sadness       0.55      0.74      0.63       187
    Surprise       0.89      0.79      0.84        62

    accuracy                           0.59      1180
   macro avg       0.61      0.63      0.61      1180
weighted avg       0.60      0.59      0.59      1180

[[142   0  11  13  19   1   1   0]
 [  0  13   1   0   0   0   2   0]
 [ 12   0  89  16  11  28  29   2]
 [ 10   0  19  90   9  11  47   1]
 [ 42   2  23  13  80  16   9   2]
 [  0   6  25  10   4  99  23   0]
 [  0   6   8  10   7  17 138   1]
 [  0   1   4   2   4   1   1  49]]


In [21]:
experiment(rf_clf,'org','ds',features_score_above_p50,'emo')

              precision    recall  f1-score   support

       Anger       0.49      0.71      0.58        62
    Calmness       0.61      0.88      0.72        16
     Disgust       0.30      0.50      0.38        62
        Fear       0.47      0.56      0.51        62
   Happiness       0.78      0.43      0.55       187
  Neutrality       0.77      0.59      0.67       167
     Sadness       0.41      0.73      0.53        62
    Surprise       0.94      0.79      0.86        62

    accuracy                           0.58       680
   macro avg       0.60      0.65      0.60       680
weighted avg       0.66      0.58      0.59       680

[[44  0  4  5  8  0  1  0]
 [ 0 14  1  0  0  0  1  0]
 [ 2  0 31  4  5  8 11  1]
 [ 1  0  5 35  1  3 17  0]
 [40  2 24 18 80 12 10  1]
 [ 2  5 28  8  3 99 22  0]
 [ 0  2  5  1  2  6 45  1]
 [ 0  0  4  4  3  0  2 49]]


In [22]:
experiment(rf_clf,'aug','ds',features_score_above_p50,'emo')

              precision    recall  f1-score   support

       Anger       0.64      0.68      0.66        62
    Calmness       0.45      0.94      0.61        16
     Disgust       0.64      0.26      0.37        62
        Fear       0.67      0.48      0.56        62
   Happiness       0.69      0.59      0.64       187
  Neutrality       0.61      0.86      0.71       167
     Sadness       0.53      0.50      0.52        62
    Surprise       0.84      0.82      0.83        62

    accuracy                           0.64       680
   macro avg       0.63      0.64      0.61       680
weighted avg       0.65      0.64      0.63       680

[[ 42   0   1   2  14   1   1   1]
 [  0  15   0   0   0   0   0   1]
 [  1   1  16   2  14  20   6   2]
 [  2   0   1  30   3  15  10   1]
 [ 21   5   4   9 110  30   4   4]
 [  0   7   3   1   7 143   6   0]
 [  0   4   0   0   4  22  31   1]
 [  0   1   0   1   7   2   0  51]]


In [23]:
experiment(rf_clf,'aug','org',features_score_above_p50,'emo')

              precision    recall  f1-score   support

       Anger       0.77      0.68      0.73       187
    Calmness       0.44      0.94      0.60        16
     Disgust       0.73      0.24      0.36       187
        Fear       0.77      0.45      0.57       187
   Happiness       0.48      0.60      0.53       187
  Neutrality       0.41      0.86      0.56       167
     Sadness       0.68      0.58      0.63       187
    Surprise       0.77      0.81      0.79        62

    accuracy                           0.58      1180
   macro avg       0.63      0.65      0.60      1180
weighted avg       0.65      0.58      0.58      1180

[[128   0   2   2  45   7   1   2]
 [  0  15   0   0   0   0   0   1]
 [  4   1  45   9  30  83  11   4]
 [  9   0   5  85  20  34  30   4]
 [ 25   4   2   9 113  28   4   2]
 [  0   6   3   1   8 143   6   0]
 [  0   7   2   3  15  49 109   2]
 [  0   1   3   1   6   1   0  50]]


In [24]:
experiment(rf_clf,'aug','org',features_score_above_p50,'sti')

              precision    recall  f1-score   support

          -1       0.83      0.75      0.79       748
           0       0.51      0.78      0.62       183
           1       0.65      0.59      0.62       249

    accuracy                           0.72      1180
   macro avg       0.67      0.71      0.68      1180
weighted avg       0.74      0.72      0.73      1180

[[560 115  73]
 [ 35 143   5]
 [ 78  23 148]]


In [28]:
experiment(rf_clf,'aug','org',features_score_above_p50,'sti')

              precision    recall  f1-score   support

          -1       0.83      0.74      0.79       748
           0       0.49      0.79      0.61       183
           1       0.66      0.59      0.62       249

    accuracy                           0.72      1180
   macro avg       0.66      0.71      0.67      1180
weighted avg       0.74      0.72      0.72      1180

[[556 120  72]
 [ 35 144   4]
 [ 75  28 146]]


In [27]:
experiment(rf_clf,'org','org',features_score_above_p50,'sti')

              precision    recall  f1-score   support

          -1       0.73      0.96      0.83       748
           0       0.80      0.38      0.52       183
           1       0.86      0.40      0.54       249

    accuracy                           0.75      1180
   macro avg       0.80      0.58      0.63      1180
weighted avg       0.77      0.75      0.72      1180

[[717  16  15]
 [112  70   1]
 [148   2  99]]


## Forward Selection by category （OLD）
- select certain category features
- remove low related features (manual study results in selected_<group>) 
- apply auto feature selection to determin how many feature chosen
- OPTIONAL experiment how much performance drop from full -> selected -> less -> 2PCs?

- Alternative is too do prosody features together AUTO

### Prodosy Features List
Total 56   
['Intensity_min_intensity', 'Intensity_relative_min_intensity_time', 'Intensity_max_intensity', 'Intensity_relative_max_intensity_time', 'Intensity_mean_intensity', 'Intensity_stddev_intensity', 'Intensity_q1_intensity', 'Intensity_median_intensity', 'Intensity_q3_intensity', 'Pitch_voiced_fraction', 'Pitch_min_pitch', 'Pitch_relative_min_pitch_time', 'Pitch_max_pitch', 'Pitch_relative_max_pitch_time', 'Pitch_mean_pitch', 'Pitch_stddev_pitch', 'Pitch_q1_pitch', 'Pitch_median_intensity', 'Pitch_q3_pitch', 'Pitch_mean_absolute_pitch_slope', 'Pitch_pitch_slope_without_octave_jumps', 'GNE_max_gne', 'GNE_mean_gne', 'GNE_stddev_gne', 'GNE_sum_gne', 'HNR_min_hnr', 'HNR_relative_min_hnr_time', 'HNR_max_hnr', 'HNR_relative_max_hnr_time', 'HNR_mean_hnr', 'HNR_stddev_hnr', 'Spectrum_band_energy', 'Spectrum_band_density', 'Spectrum_band_energy_difference', 'Spectrum_band_density_difference', 'Spectrum_center_of_gravity_spectrum', 'Spectrum_stddev_spectrum', 'Spectrum_skewness_spectrum', 'Spectrum_kurtosis_spectrum', 'Spectrum_central_moment_spectrum', 'Formant_f1_mean', 'Formant_f2_mean', 'Formant_f3_mean', 'Formant_f4_mean', 'Formant_f1_median', 'Formant_f2_median', 'Formant_f3_median', 'Formant_f4_median', 'Formant_formant_dispersion', 'Formant_average_formant', 'Formant_mff', 'Formant_fitch_vtl', 'Formant_delta_f', 'Formant_vtl_delta_f', 'Local Jitter', 'Local Shimmer']

In [ ]:
feature_names_prosody = ['Intensity_min_intensity', 'Intensity_relative_min_intensity_time', 'Intensity_max_intensity', 'Intensity_relative_max_intensity_time', 'Intensity_mean_intensity', 'Intensity_stddev_intensity', 'Intensity_q1_intensity', 'Intensity_median_intensity', 'Intensity_q3_intensity', 
                         'Pitch_voiced_fraction', 'Pitch_min_pitch', 'Pitch_relative_min_pitch_time', 'Pitch_max_pitch', 'Pitch_relative_max_pitch_time', 'Pitch_mean_pitch', 'Pitch_stddev_pitch', 'Pitch_q1_pitch', 'Pitch_median_intensity', 'Pitch_q3_pitch', 'Pitch_mean_absolute_pitch_slope', 'Pitch_pitch_slope_without_octave_jumps', 
                         'GNE_max_gne', 'GNE_mean_gne', 'GNE_stddev_gne', 'GNE_sum_gne', 'HNR_min_hnr', 'HNR_relative_min_hnr_time', 'HNR_max_hnr', 'HNR_relative_max_hnr_time', 'HNR_mean_hnr', 'HNR_stddev_hnr', 
                         'Spectrum_band_energy', 'Spectrum_band_density', 'Spectrum_band_energy_difference', 'Spectrum_band_density_difference', 'Spectrum_center_of_gravity_spectrum', 'Spectrum_stddev_spectrum', 'Spectrum_skewness_spectrum', 'Spectrum_kurtosis_spectrum', 'Spectrum_central_moment_spectrum', 'Formant_f1_mean', 'Formant_f2_mean', 'Formant_f3_mean', 'Formant_f4_mean', 'Formant_f1_median', 'Formant_f2_median', 'Formant_f3_median', 'Formant_f4_median', 'Formant_formant_dispersion', 'Formant_average_formant', 'Formant_mff', 'Formant_fitch_vtl', 'Formant_delta_f', 'Formant_vtl_delta_f', 
                         'Local Jitter', 'Local Shimmer']
print(len(feature_names_prosody))
# feature_names_prosody_selected = [ name for name in feature_names_prosody if name in features_score_above_p50 ]
# selected_spect = ['Spectrum_band_energy_difference','Spectrum_band_density_difference','Spectrum_center_of_gravity_spectrum','Spectrum_skewness_spectrum','Spectrum_kurtosis_spectrum', 'Spectrum_stddev_spectrum','Spectrum_band_density', 'Spectrum_band_energy']
# selected_formant = ['Formant_f1_mean','Formant_f1_median','Formant_f3_mean','Formant_fitch_vtl','Formant_mff','Formant_formant_dispersion']
# selected_GNE = ['GNE_max_gne', 'GNE_stddev_gne', 'GNE_mean_gne', 'GNE_sum_gne']
# selected_pitch = ['Pitch_pitch_slope_without_octave_jumps', 'Pitch_q3_pitch','Pitch_stddev_pitch', 'Pitch_mean_absolute_pitch_slope','Pitch_mean_pitch', 'Pitch_max_pitch', 'Pitch_q1_pitch', 'Pitch_min_pitch']
# selected_intensity = ['Intensity_max_intensity', 'Intensity_q3_intensity','Intensity_median_intensity', 'Intensity_mean_intensity', 'Intensity_stddev_intensity','Intensity_relative_max_intensity_time']
# selected_HNR = ['HNR_stddev_hnr', 'HNR_mean_hnr','HNR_relative_min_hnr_time','HNR_max_hnr']
# feature_names_prosody_selected = selected_spect + selected_formant + selected_HNR + selected_intensity + selected_pitch + ['Local Jitter','Local Shimmer']
# print(len(feature_names_prosody_selected))

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

rf_est = RandomForestClassifier(n_estimators=25, random_state=42)
sfs = SequentialFeatureSelector(estimator=rf_est, 
                                n_features_to_select='auto',  # Select one feature at a time
                                tol = 0.001, # minimum amount incremented between two consecutive additions,
                                direction='forward',  # Forward selection
                                scoring='balanced_accuracy',  # Use balabced accuracy as the evaluation metric - average of recall
                                cv=5)  # k-fold cross-validation

In [ ]:
def forward_select(group_name):
    start = time.time()
    features = [ name for name in feature_names_prosody if group_name in name ]
    X_train = df_joint_train[features]
    X_test  = df_joint_test[features]
    sfs.fit(X_train, y_train_s)

    selected_feature_names = [features[i] for i, selected in enumerate(sfs.get_support()) if selected]
    print(f"{len(selected_feature_names)} of {len(features)} selected. \nFeature names:", selected_feature_names)
    X_train_selected = sfs.transform(X_train)
    X_test_selected = sfs.transform(X_test)
    rf_est.fit(X_train_selected, y_train_s)
    accuracy = rf_est.score(X_test_selected, y_test_s)
    print("Accuracy with selected features:", accuracy)
    print(f"time taken: {round(time.time()-start)} seconds.")
    return selected_feature_names

# forward_select('Intensity')

In [ ]:
selected_prosody = []
for group in ['Pitch','Intensity','HNR','Spectrum','Formant']:
    selected_prosody.extend(forward_select(group))

In [ ]:
len(selected_prosody), selected_prosody

In [ ]:
try_clf_with_feature_selected(dt_clf,selected_prosody,y_train_s,y_test_s)

## now try backwar/d

In [ ]:
def backward_select(group_name):
    start = time.time()
    features = [ name for name in feature_names_prosody if group_name in name ]
    X_train = df_joint_train[features]
    X_test  = df_joint_test[features]
    sfs_bk = SequentialFeatureSelector(estimator=rf_est, 
                                n_features_to_select='auto',  # Select one feature at a time
                                direction='backward',  # Forward selection
                                scoring='balanced_accuracy',  # Use balabced accuracy as the evaluation metric - average of recall
                                cv=5)  # k-fold cross-validation
    sfs_bk.fit(X_train, y_train_s)

    selected_feature_names = [features[i] for i, selected in enumerate(sfs_bk.get_support()) if selected]
    print(f"{len(selected_feature_names)} of {len(features)} selected. \nFeature names:", selected_feature_names)
    X_train_selected = sfs_bk.transform(X_train)
    X_test_selected = sfs_bk.transform(X_test)
    rf_est.fit(X_train_selected, y_train_s)
    accuracy = rf_est.score(X_test_selected, y_test_s)
    print("Accuracy with selected features:", accuracy)
    print(f"time taken: {round(time.time()-start)} seconds.")
    return selected_feature_names

# backward_select('Pitch')

In [ ]:
selected_prosody_bk = []
for group in ['Pitch','Intensity','HNR','Spectrum','Formant']:
    selected_prosody_bk.extend(backward_select(group))
print(len(selected_prosody_bk), selected_prosody_bk)
try_clf_with_feature_selected(dt_clf,selected_prosody_bk,y_train_s,y_test_s)

## simple filter based on ranking -result 

In [ ]:
selected_pitch = ['Pitch_pitch_slope_without_octave_jumps', 'Pitch_q3_pitch','Pitch_stddev_pitch', 
                  'Pitch_mean_absolute_pitch_slope','Pitch_mean_pitch', 'Pitch_max_pitch', 'Pitch_q1_pitch', 'Pitch_min_pitch']
selected_intensity = ['Intensity_max_intensity', 'Intensity_q3_intensity','Intensity_median_intensity', 
                      'Intensity_mean_intensity', 'Intensity_stddev_intensity','Intensity_relative_max_intensity_time']
selected_spect = ['Spectrum_band_energy_difference','Spectrum_band_density_difference','Spectrum_center_of_gravity_spectrum',
                  'Spectrum_skewness_spectrum','Spectrum_kurtosis_spectrum', 'Spectrum_stddev_spectrum','Spectrum_band_density', 'Spectrum_band_energy']
selected_formant = ['Formant_f1_mean','Formant_f1_median','Formant_f3_mean','Formant_fitch_vtl','Formant_mff','Formant_formant_dispersion']
selected_HNR = ['HNR_stddev_hnr', 'HNR_mean_hnr','HNR_relative_min_hnr_time','HNR_max_hnr']
feature_names_prosody_filtered= selected_spect + selected_formant + selected_HNR + selected_intensity + selected_pitch + ['Local Jitter','Local Shimmer']
len(feature_names_prosody_filtered)

## try backward selection 

In [ ]:
feature_names_prosody = ['Intensity_min_intensity', 'Intensity_relative_min_intensity_time', 'Intensity_max_intensity',
                         'Intensity_relative_max_intensity_time', 'Intensity_mean_intensity', 'Intensity_stddev_intensity', 
                         'Intensity_q1_intensity', 'Intensity_median_intensity', 'Intensity_q3_intensity', 
                         'Pitch_voiced_fraction', 'Pitch_min_pitch', 'Pitch_relative_min_pitch_time', 'Pitch_max_pitch', 
                         'Pitch_relative_max_pitch_time', 'Pitch_mean_pitch', 'Pitch_stddev_pitch', 'Pitch_q1_pitch', 
                         'Pitch_median_intensity', 'Pitch_q3_pitch', 'Pitch_mean_absolute_pitch_slope', 'Pitch_pitch_slope_without_octave_jumps',
                         'HNR_min_hnr', 'HNR_relative_min_hnr_time', 'HNR_max_hnr', 'HNR_relative_max_hnr_time', 'HNR_mean_hnr', 'HNR_stddev_hnr', 
                         'Spectrum_band_energy', 'Spectrum_band_density', 'Spectrum_band_energy_difference', 'Spectrum_band_density_difference', 'Spectrum_center_of_gravity_spectrum', 
                         'Spectrum_stddev_spectrum', 'Spectrum_skewness_spectrum', 'Spectrum_kurtosis_spectrum', 'Spectrum_central_moment_spectrum', 
                         'Formant_f1_mean', 'Formant_f2_mean', 'Formant_f3_mean', 'Formant_f4_mean', 'Formant_f1_median', 
                         'Formant_f2_median', 'Formant_f3_median', 'Formant_f4_median', 'Formant_formant_dispersion', 'Formant_average_formant', 
                         'Formant_mff', 'Formant_fitch_vtl', 'Formant_delta_f', 'Formant_vtl_delta_f', 
                         'Local Jitter', 'Local Shimmer']

In [ ]:
start = time.time()
X_train = df_joint_train[feature_names_prosody]
X_test  = df_joint_test[feature_names_prosody]
rf_est = RandomForestClassifier(n_estimators=25)
sfs_bk = SequentialFeatureSelector(estimator=rf_est, 
                            n_features_to_select='auto',  # Select one feature at a time
                            direction='backward',  # Forward selection
                            scoring='balanced_accuracy',  # Use balabced accuracy as the evaluation metric - average of recall
                            cv=3)  # k-fold cross-validation
sfs_bk.fit(X_train, y_train_s)

selected_feature_names = [feature_names_prosody[i] for i, selected in enumerate(sfs_bk.get_support()) if selected]
print(f"{len(selected_feature_names)} of {len(feature_names_prosody)} selected. \nFeature names:", selected_feature_names)
X_train_selected = sfs_bk.transform(X_train)
X_test_selected = sfs_bk.transform(X_test)
rf_est.fit(X_train_selected, y_train_s)
accuracy = rf_est.score(X_test_selected, y_test_s)
print("Accuracy with selected features:", accuracy)
print(f"time taken: {round(time.time()-start)} seconds.")
print("selected: ", selected_feature_names)

In [ ]:
try_clf_with_feature_selected(rf_clf,feature_names_prosody,y_train_s,y_test_s)
try_clf_with_feature_selected(rf_clf,selected_feature_names,y_train_s,y_test_s)

In [ ]:
try_clf_with_feature_selected(rf_clf,feature_names_prosody,y_train_e,y_test_e)
try_clf_with_feature_selected(rf_clf,selected_feature_names,y_train_e,y_test_e)

In [ ]:
start = time.time()
sfs = SequentialFeatureSelector(estimator=rf_est, 
                            n_features_to_select='auto',  # Select one feature at a time
                            direction='forward',  # Forward selection
                            scoring='balanced_accuracy',  # Use balabced accuracy as the evaluation metric - average of recall
                            cv=5)  # k-fold cross-validation
sfs.fit(X_train, y_train_s)

selected_feature_names = [feature_names_prosody[i] for i, selected in enumerate(sfs.get_support()) if selected]
print(f"{len(selected_feature_names)} of {len(feature_names_prosody)} selected. \nFeature names:", selected_feature_names)
X_train_selected = sfs.transform(X_train)
X_test_selected = sfs.transform(X_test)
rf_est.fit(X_train_selected, y_train_s)
accuracy = rf_est.score(X_test_selected, y_test_s)
print("Accuracy with selected features:", accuracy)
print(f"time taken: {round(time.time()-start)} seconds.")
print("selected: ", selected_feature_names)

difference checking  
[ 'Intensity_max_intensity',  'Pitch_relative_max_pitch_time',  'Pitch_median_intensity',  'Pitch_mean_absolute_pitch_slope',   
'Spectrum_skewness_spectrum', 'Spectrum_central_moment_spectrum', 
  'Formant_f1_median', 'Formant_f4_median', 'Formant_formant_dispersion', 'Formant_average_formant', 'Formant_delta_f',]
[ 'Intensity_relative_max_intensity_time',   'Pitch_min_pitch', 'Pitch_max_pitch',   'Pitch_q1_pitch',   
 'HNR_min_hnr',  'HNR_max_hnr', 'HNR_mean_hnr', 'Formant_f2_median'
 'Spectrum_band_density', 'Spectrum_band_energy_difference','Spectrum_center_of_gravity_spectrum', 'Spectrum_skewness_spectrum', ]

17 of 34 selected.   
Feature names: ['Spectrum_center_of_gravity_spectrum', 'Spectrum_stddev_spectrum', 'Spectrum_band_density', 'Spectrum_band_energy', 'Formant_f1_mean', 'Formant_mff', 'HNR_stddev_hnr', 'HNR_mean_hnr', 'HNR_relative_min_hnr_time', 'Intensity_q3_intensity', 'Intensity_relative_max_intensity_time', 'Pitch_pitch_slope_without_octave_jumps', 'Pitch_stddev_pitch', 'Pitch_mean_pitch', 'Pitch_max_pitch', 'Pitch_q1_pitch', 'Local Jitter']  
Accuracy with selected features: 0.7694915254237288  
time taken: 2287 seconds.  
selected:  ['Spectrum_center_of_gravity_spectrum', 'Spectrum_stddev_spectrum', 'Spectrum_band_density', 'Spectrum_band_energy', 'Formant_f1_mean', 'Formant_mff', 'HNR_stddev_hnr', 'HNR_mean_hnr', 'HNR_relative_min_hnr_time', 'Intensity_q3_intensity', 'Intensity_relative_max_intensity_time', 'Pitch_pitch_slope_without_octave_jumps', 'Pitch_stddev_pitch', 'Pitch_mean_pitch', 'Pitch_max_pitch', 'Pitch_q1_pitch', 'Local Jitter']  

# End

### Feature Tables Part 2 

| feature category   | num of sub attributes  |
| ------------------ | ---------------------- |
|Intensity           |    9                   |
|pitch               |   12                   |
|HNR                 |    6                   |
|GNE                 |    5                   |
|Local Jitter        |    1                   |
|Local Shimmer       |    1                   |
|Spectrum            |    9                   | 
|formant             |   14                   |